In [4]:
import pandas as pd
from ydata_profiling import ProfileReport
from modulos.cleaning import *

In [17]:
url = "https://drive.google.com/file/d/1mzY4SYyh-VJneRTF7zc-8PgYatQRQnX8/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)
df.head()

,event_name,item_id,timestamp,site,experiments,user_id
0,SEARCH,NaN,2021-08-02T23:55:38.966-0400,MLA,"{searchbackend/recommended-products=6157, mcli...",3204901
1,PRODUCT,882352139.0,2021-08-02T23:55:51.673-0400,MLA,"{qadb/sa-on-vip=6695, vip/showV2V3BoxMessages=...",3204901
2,PRODUCT,655266729.0,2021-08-02T23:56:16.083-0400,MLA,"{qadb/sa-on-vip=6695, vip/showV2V3BoxMessages=...",3204901
3,PRODUCT,761520929.0,2021-08-02T23:56:29.989-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",3204901
4,PRODUCT,757586409.0,2021-08-02T23:56:47.887-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",3204901


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141553 entries, 0 to 141552
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   event_name   141553 non-null  object 
 1   item_id      64803 non-null   float64
 2   timestamp    141553 non-null  object 
 3   site         141553 non-null  object 
 4   experiments  141553 non-null  object 
 5   user_id      141553 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 6.5+ MB


In [28]:
df.event_name.value_counts(normalize=True) * 100

event_name
SEARCH        54.219974
PRODUCT       41.358360
CHECKOUT_1     1.494140
CHECKOUT_2     1.261718
CHECKOUT_3     0.897190
BUY            0.768617
Name: proportion, dtype: float64

In [19]:
df_experiments = df['experiments'].str.replace('{','').str.replace('}','').str.split(', ', expand=True)
df_all = pd.concat([df, df_experiments], axis=1)
df_melted = pd.melt(df_all, id_vars=['event_name',         'item_id',       'timestamp',
                  'site',           'user_id'], value_vars=[0,                 1,
                       2,                 3,                 4,
                       5,                 6,                 7,
                       8,                 9,                10,
                      11,                12,                13,
                      14,                15,                16,
                      17,                18],
                    var_name='Variable', value_name='experiment')
df_variant = df_melted.experiment.str.split('=', expand=True)
df_melted = pd.concat([df_melted, df_variant], axis=1)
df_melted.drop(columns=['experiment','Variable'], inplace=True)
df_melted.columns = ['event_name', 'item_id', 'timestamp', 'site', 'user_id', 'experiment', 'variante']
df_melted = df_melted.dropna(subset=['experiment'])

In [20]:
df_melted.head()

,event_name,item_id,timestamp,site,user_id,experiment,variante
0,SEARCH,NaN,2021-08-02T23:55:38.966-0400,MLA,3204901,searchbackend/recommended-products,6157
1,PRODUCT,882352139.0,2021-08-02T23:55:51.673-0400,MLA,3204901,qadb/sa-on-vip,6695
2,PRODUCT,655266729.0,2021-08-02T23:56:16.083-0400,MLA,3204901,qadb/sa-on-vip,6695
3,PRODUCT,761520929.0,2021-08-02T23:56:29.989-0400,MLA,3204901,search/remove-ecn-tag,4954
4,PRODUCT,757586409.0,2021-08-02T23:56:47.887-0400,MLA,3204901,search/remove-ecn-tag,4954


In [21]:
df_melted.experiment.nunique()

47

In [22]:
print(f"Se tienen {df_melted.experiment.nunique()} experimentos")

Se tienen 47 experimentos


In [24]:
profile_melted = ProfileReport(df_melted, title="Informe del Dataset Iris", explorative=True)

# Visualizar el informe
profile_melted.to_notebook_iframe()

Render HTML: 100%|██████████| 1/1 [00:00<00:00,  7.28it/s]


# Insights
- Todos los items nulos son del evento **search**
- Hay experimentos que pertenecen a más de un evento
- Hay filas duplicadas y también filas con la misma información pero con diferencias de tiempo muy pequeñas (menos de un minuto)

# Supuestos
- La granularidad de la información será experimento y variante, por fecha (A-M-D H), evento, usuario e item. 
- Si la diferencia entre los registros es menor a un minuto y contienen la misma información, se conservará solo un registro.
- Para mirar que experimentos llevaron a la compra se tomo el ID y Usario de los eventos BUY, creando un identificador combinado. Luego se mira en que otros eventos se tiene este mismo identificador, y así se marcan los experimentos llevaron a una compra en cada uno de los eventos.
- Se considera el flujo de los eventos como Search, Product, Checkout1, Checkout2, Checkout3 y Buy, aunque es claro que los eventos no necesariamente siguen este orden.
- El supuesto anterior se aplica para los eventos Search, ya que al no tener item_id, no se podían asociar las comprar. Para este caso se valida si el evento siguiente es marcado con compra. En ese caso la búsqueda también se marcará con compra.
- Para la limpieza del dataset se analizaron eventos previos y siguientes a cada registro